# Goal

The objective of this notebook is to analyze the impact of maintaining and using a tentative pareto.

# Method
We will start simple: optimizing a 2D and 5D hypersphere 10 times each with and without tentative pareto. Then I'll plot pareto volume as a function of number of iterations. 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import concurrent.futures

from run_optimization import run_optimization

num_runs = 10
volume_estimators_over_time_by_strategy = {True:[], False:[]}

with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    outstanding_futures = set()
    
    for i in range(num_runs):
        add_pending_suggestions = (i % 2 == 0)
        future = executor.submit(run_optimization, i, 10, 200, add_pending_suggestions)
        outstanding_futures.add(future)
        
    done_futures, outstanding_futures = concurrent.futures.wait(outstanding_futures, return_when=concurrent.futures.ALL_COMPLETED)
    for future in done_futures:
        add_pending_suggestions, pareto_volume_estimators_over_time = future.result()
        volume_estimators = volume_estimators_over_time_by_strategy[add_pending_suggestions]
        volume_estimators.append(pareto_volume_estimators_over_time)
        

In [ ]:
import plotly.graph_objs as go
fig = go.Figure()

runs = volume_estimators_over_time_by_strategy[True]
num_runs = len(runs)
num_data_points_per_run = len(runs[0])
total_volume_over_time_with_tentative_pareto = [0 for run in range(num_data_points_per_run)]
for run in runs:
    current_run = []
    for i, estimator in enumerate(run):
        lower_bound, upper_bound = estimator.get_two_sided_confidence_interval_on_pareto_volume()
        center = (lower_bound + upper_bound) / 2
        current_run.append(center)
        total_volume_over_time_with_tentative_pareto[i] += center
    
    fig.add_trace(go.Scatter(
        y=current_run,
        name=f'run_{i}_with_tentative_pareto',
        marker={'color':'red'}
    ))
        
average_volume_over_time_with_tentative_pareto = [total / num_runs for total in total_volume_over_time_with_tentative_pareto]

fig.add_trace(go.Scatter(
    y=average_volume_over_time_with_tentative_pareto,
    name='average_volume_over_time_with_tentative_pareto',
    marker={'color':'red'}
))

runs = volume_estimators_over_time_by_strategy[False]
num_runs = len(runs)
num_data_points_per_run = len(runs[0])
total_volume_over_time_without_tentative_pareto = [0 for run in range(num_data_points_per_run)]
for run in runs:
    current_run = []
    for i, estimator in enumerate(run):
        lower_bound, upper_bound = estimator.get_two_sided_confidence_interval_on_pareto_volume()
        center = (lower_bound + upper_bound) / 2
        current_run.append(center)
        total_volume_over_time_without_tentative_pareto[i] += center
    
    fig.add_trace(go.Scatter(
        y=current_run,
        name=f'run_{i}_without_tentative_pareto',
        marker={'color':'blue'}
    ))
        
average_volume_over_time_without_tentative_pareto = [total / num_runs for total in total_volume_over_time_without_tentative_pareto]

fig.add_trace(go.Scatter(
    y=average_volume_over_time_without_tentative_pareto,
    name='average_volume_over_time_without_tentative_pareto',
    marker={'color':'blue'}
))

fig.show()